In [1]:
import numpy as np
import os
import commit
from commit import trk2dictionary

/Users/xinyi/Documents/GitHub/rCOMMIT/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
os.system( 'tck2connectome -force -nthreads 0 -assignment_radial_search 2 -out_assignments fibers_assignment.txt demo01_data/demo01_fibers.tck demo01_data/GM_1x1x1.nii.gz connectome.csv' )


tck2connectome: [WARNING] existing output files will be overwritten
tck2connectome: Image "demo01_data/GM_1x1x1.nii.gz" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] uncompressing image "demo01_data/GM_1x1x1.nii.gz"[0K
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [100%] uncompressing image "demo01_data/GM_1x1x1.nii.gz"[0K
tck2connectome: [100%] Constructing connectome[0K


0

In [5]:
if not os.path.isdir( 'bundles' ) :
    os.mkdir( 'bundles' )
os.system( 'connectome2tck -force -nthreads 0 -exclusive -files per_edge -keep_self demo01_data/demo01_fibers.tck fibers_assignment.txt bundles/bundle_' )

C = np.loadtxt( 'connectome.csv', delimiter=',' ) # NB: change 'delimiter' to suits your needs
CMD = 'tckedit -force -nthreads 0'
for i in range(C.shape[0]):
    CMD_i = 'tckedit -force -nthreads 0'
    for j in range(i,C.shape[0]):
        if C[i,j] > 0 :
            CMD_i += ' bundles/bundle_%d-%d.tck' %(i+1,j+1)
    os.system( CMD_i + ' bundles/demo01_fibers_connecting_%d.tck' % (i+1) )
    CMD += ' bundles/demo01_fibers_connecting_%d.tck' %(i+1)
os.system( CMD + ' demo01_fibers_connecting.tck' )

connectome2tck: [WARNING] existing output files will be overwritten
connectome2tck: [done] reading streamline assignments file[0K
connectome2tck: [WARNING] List of nodes of interest not provided; -exclusive option will have no effect
connectome2tck: [100%] Extracting tracks from connectome[0K
tckedit: [WARNING] existing output files will be overwritten
tckedit: [done]     3304 read,     3304 written
tckedit: [WARNING] existing output files will be overwritten
tckedit: [done]     1818 read,     1818 written
tckedit: [WARNING] existing output files will be overwritten
tckedit: [done]      336 read,      336 written
tckedit: [WARNING] existing output files will be overwritten
tckedit: [done]     2043 read,     2043 written
tckedit: [WARNING] existing output files will be overwritten
tckedit: [done]     2548 read,     2548 written
tckedit: [WARNING] existing output files will be overwritten
tckedit: [done]     2638 read,     2638 written
tckedit: [WARNING] existing output files will be ove

0

In [6]:
trk2dictionary.run(
    filename_tractogram = 'demo01_fibers_connecting.tck',
    filename_mask       = 'demo01_data/WM.nii.gz',
    fiber_shift         = 0.5
)

In [ ]:
## Initialize with commit

# convert the bvals/bvecs pair to a single scheme file
import amico
amico.util.fsl2scheme( 'demo01_data/bvals.txt', 'demo01_data/bvecs.txt', 'DWI.scheme' )

# load the data
mit = commit.Evaluation( '.', '.' )
mit.load_data( 'demo01_data/DWI.nii.gz', 'DWI.scheme' )

# use a forward-model with 1 Stick for the streamlines and 2 Balls for all the rest
mit.set_model( 'StickZeppelinBall' )
d_par       = 1.7E-3             # Parallel diffusivity [mm^2/s]
d_perps_zep = []                 # Perpendicular diffusivity(s) [mm^2/s]
d_isos      = [ 1.7E-3, 3.0E-3 ] # Isotropic diffusivity(s) [mm^2/s]
mit.model.set( d_par, d_perps_zep, d_isos )

mit.generate_kernels( regenerate=True )
mit.load_kernels()

# create the sparse data structures to handle the matrix A
mit.load_dictionary( 'COMMIT' )
mit.set_threads()
mit.build_operator()

# perform the fit
mit.fit( tol_fun=1e-3, max_iter=1000, verbose=False )
mit.save_results( path_suffix="_COMMIT1" )